In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

try:
    env_file = Path('.env')
    if env_file.exists():
        load_dotenv(dotenv_path=env_file, override=True)
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in .env file")
        print(f"Gemini API key loaded from .env file (first 10 chars: {google_api_key[:10]}...)")
        print(f"Verified: GOOGLE_API_KEY is set in environment (length: {len(google_api_key)} chars)")
        print("ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")


Gemini API key loaded from .env file (first 10 chars: AIzaSyCxBj...)
Verified: GOOGLE_API_KEY is set in environment (length: 39 chars)
ADK will use GOOGLE_API_KEY


In [2]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("ADK components imported successfully.")


ADK components imported successfully.


In [3]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    print(f"\n### Session: {session_id}")

    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    if isinstance(user_queries, str):
        user_queries = [user_queries]

    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("Helper functions defined.")


Helper functions defined.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)


In [5]:
memory_service = InMemoryMemoryService()


In [6]:
APP_NAME = "RecipeAssistantApp"
USER_ID = "demo_user"

recipe_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="RecipeAssistantAgent",
    instruction="Answer user questions about recipes and dietary preferences in simple words.",
)

print("Agent created")


Agent created


In [7]:
session_service = InMemorySessionService()

runner = Runner(
    agent=recipe_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("Agent and Runner created with memory support.")


Agent and Runner created with memory support.


In [8]:
await run_session(
    runner,
    "I am vegetarian and I love Italian cuisine. Can you suggest a pasta recipe?",
    "conversation-01",
)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



### Session: conversation-01

User > I am vegetarian and I love Italian cuisine. Can you suggest a pasta recipe?
Model: > Sure! How about a delicious **Pasta Primavera**?

It's a vegetarian Italian classic! It means "spring pasta" and is usually made with lots of fresh vegetables. You can use whatever veggies you have on hand, like:

*   Broccoli
*   Zucchini
*   Bell peppers
*   Asparagus
*   Peas
*   Carrots

You'll typically cook pasta, then sauté the vegetables with garlic and olive oil. Everything is tossed together with some herbs and maybe a little bit of cream or a light sauce. It's super easy and healthy!

Does that sound good?


In [9]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

print("Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")


Session contains:
  user: I am vegetarian and I love Italian cuisine. Can you suggest ...
  model: Sure! How about a delicious **Pasta Primavera**?

It's a veg...


In [10]:
await memory_service.add_session_to_memory(session)

print("Session added to memory.")


Session added to memory.


In [11]:
recipe_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="RecipeAssistantAgent",
    instruction="Answer user questions about recipes and dietary preferences. Use load_memory tool if you need to recall past conversations about dietary restrictions or cuisine preferences.",
    tools=[load_memory],
)

print("Agent with load_memory tool created.")


Agent with load_memory tool created.


In [12]:
runner = Runner(
    agent=recipe_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "What cuisine do I prefer?", "preference-test")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



### Session: preference-test

User > What cuisine do I prefer?


In [13]:
await run_session(runner, "I am allergic to peanuts and tree nuts.", "dietary-session-01")



### Session: dietary-session-01

User > I am allergic to peanuts and tree nuts.
Model: > Thanks for letting me know. I'll be sure to avoid recipes with peanuts and tree nuts.


In [14]:
dietary_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="dietary-session-01"
)

await memory_service.add_session_to_memory(dietary_session)

print("Dietary session saved to memory.")


Dietary session saved to memory.


In [15]:
await run_session(
    runner, "What are my dietary restrictions?", "dietary-session-02"
)



### Session: dietary-session-02

User > What are my dietary restrictions?


In [16]:
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What are the user's dietary restrictions?"
)

print("Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")


Search Results:
  Found 1 relevant memories

  [RecipeAssistantAgent]: Sure! How about a delicious **Pasta Primavera**?

It's a vegetarian Italian clas...


In [17]:
async def auto_save_to_memory(callback_context):
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("Callback created.")


Callback created.


In [18]:
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoMemoryRecipeAgent",
    instruction="Answer user questions about recipes and dietary preferences.",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,
)

print("Agent created with automatic memory saving.")


Agent created with automatic memory saving.


In [19]:
auto_runner = Runner(
    agent=auto_memory_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("Runner created.")


Runner created.


In [20]:
await run_session(
    auto_runner,
    "I prefer spicy food and I love Thai cuisine!",
    "auto-save-test",
)

await run_session(
    auto_runner,
    "What type of cuisine do I like?",
    "auto-save-test-2",
)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



### Session: auto-save-test

User > I prefer spicy food and I love Thai cuisine!
Model: > Great! How about a spicy **Thai Green Curry**?

It's a flavorful and aromatic dish that can easily be made vegetarian and nut-free. The key ingredients usually include:

*   **Green curry paste:** This is where the spice comes from! You can adjust the amount to control the heat.
*   **Coconut milk:** For a creamy base.
*   **Vegetables:** Think bamboo shoots, bell peppers, zucchini, eggplant, and green beans.
*   **Protein:** Tofu is a fantastic vegetarian option.
*   **Thai basil and cilantro:** For fresh, vibrant flavor.

It's typically served with jasmine rice.

Does a spicy Thai Green Curry sound appealing to you?

### Session: auto-save-test-2

User > What type of cuisine do I like?
Model: > You like Italian and Thai cuisine. You also mentioned that you prefer spicy food.


In [21]:
await run_session(
    auto_runner,
    "I also enjoy Mexican food and prefer medium spice level.",
    "auto-save-test-3",
)

await run_session(
    auto_runner,
    "Tell me a joke about cooking.",
    "auto-save-test-4",
)



### Session: auto-save-test-3

User > I also enjoy Mexican food and prefer medium spice level.
Model: > Great! Considering your preference for Mexican cuisine and medium spice levels, how about a **Vegetarian Enchiladas Verdes**?

This dish is a fantastic choice because:

*   It's a classic Mexican comfort food.
*   You can easily control the spice level by adjusting the amount of jalapeños or serrano peppers in the salsa verde.
*   It's very versatile with vegetarian fillings.

Common ingredients include:

*   **Corn tortillas:** For wrapping.
*   **Filling:** Black beans, corn, sautéed onions and peppers, and perhaps some crumbled seasoned tofu or a vegetarian cheese blend.
*   **Salsa Verde:** Made with tomatillos, cilantro, onion, garlic, and your choice of chiles (like jalapeños for medium heat).
*   **Toppings:** Avocado, cilantro, a dollop of sour cream or vegan crema.

It's baked until bubbly and delicious.

Does Vegetarian Enchiladas Verdes sound like something you'd enjoy?



In [22]:
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="cuisine preferences"
)

print("Final Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:100]
        print(f"  [{memory.author}]: {text}...")


Final Search Results:
  Found 5 relevant memories

  [user]: I am vegetarian and I love Italian cuisine. Can you suggest a pasta recipe?...
  [user]: I prefer spicy food and I love Thai cuisine!...
  [user]: What type of cuisine do I like?...
  [AutoMemoryRecipeAgent]: You like Italian and Thai cuisine. You also mentioned that you prefer spicy food....
  [AutoMemoryRecipeAgent]: Great! Considering your preference for Mexican cuisine and medium spice levels, how about a **Vegeta...
